In [2]:
import numpy as np 
import matplotlib.pyplot as plt
from scipy import stats

### No Agent Version

This selects the optimal time based solely off distributions, it's as if you can't see the agent, but if you go after them on the reaction time 

In [35]:
# Not including probability of agent, only if probability of reacting and making it drops below 0.5
reaction_movement_time = 350
reaction_movement_uncertainty = 40 # We have some uncertainty about making it 
movement_uncertainty = 20
movement_time = 125

timing_uncertainty = 75 # Ok so as we iterate thru every time step, we have some distribution around the actual time

time_steps = np.arange(0,1510,10)
trials =100
uncertain_decision_time = np.zeros((trials))
actual_decision_time = np.zeros((trials))
for i in range(trials):
    j=-1
    for time in time_steps:
        j+=1
        t = stats.norm.rvs(time_steps[j],timing_uncertainty)
        prob_of_reacting_and_making = stats.norm.cdf(1500-t,reaction_movement_time,reaction_movement_uncertainty)
        prob_of_making = stats.norm.cdf(1500-t,movement_time,movement_uncertainty)
        if prob_of_reacting_and_making<=0.5:
            print(prob_of_reacting_and_making,prob_of_making)
            uncertain_decision_time[i] = t
            actual_decision_time[i] = time
            break

0.012477484848483617 0.9999999999933713
0.13228996939805526 1.0
0.03397609138705086 0.9999999999999851
0.47120037612988147 1.0
0.21535500345464142 1.0
0.08471312796065966 1.0
0.2446039891663846 1.0
0.16958065307037462 1.0
0.26681186504216337 1.0
0.30683020932476585 1.0
0.19647744882027668 1.0
0.027384173917767037 0.9999999999999362
0.20721232531992073 1.0
0.1354558180767399 1.0
0.26900522258638593 1.0
0.23975482840515344 1.0
0.47933620834198354 1.0
0.028742388248529482 0.9999999999999536
0.23590177653894912 1.0
0.039690408575449025 0.9999999999999951
0.23953960620011372 1.0
0.30569174199453913 1.0
0.4427886687165207 1.0
0.41739959335088883 1.0
0.4490561674609995 1.0
0.2402315335704891 1.0
0.10349852999483838 1.0
0.09653450782760215 1.0
0.4419703745855455 1.0
0.25969459074831475 1.0
0.2978171432030918 1.0
0.17241933515780855 1.0
0.22686774032330942 1.0
0.4130309708652003 1.0
0.08927748715776362 1.0
0.3859977932978721 1.0
0.3038632105974012 1.0
0.36807993792850957 1.0
0.2325957942142941 

In [47]:
# Now throw in the prior of prob_of_using_agent
agent_mean_decision_time, agent_sd_decision_time = 1100,50
prob_of_using_agent = stats.norm.cdf(1500,agent_mean_decision_time + reaction_movement_time,agent_sd_decision_time)
reaction_movement_time = 350
reaction_movement_uncertainty = 40 # We have some uncertainty about making it 
movement_uncertainty = 20
movement_time = 125

timing_uncertainty = 75 # Ok so as we iterate thru every time step, we have some distribution around the actual time

time_steps = np.arange(0,1510,10)
trials =100
uncertain_decision_time = np.zeros((trials))
actual_decision_time = np.zeros((trials))

react_reward = 1
gamble_reward = 0.5
for i in range(trials):
    j=-1
    for time in time_steps:
        j+=1
        t = stats.norm.rvs(time_steps[j],timing_uncertainty)
        prob_of_reacting_and_making = stats.norm.cdf(1500-t,reaction_movement_time,reaction_movement_uncertainty)
        expected_reward_react = prob_of_reacting_and_making*prob_of_using_agent*react_reward
        
        prob_of_making = stats.norm.cdf(1500-t,movement_time,movement_uncertainty)
        expected_reward_gamble = prob_of_making*gamble_reward
        
        if expected_reward_react<=expected_reward_gamble:
            print(expected_reward_react,prob_of_making)
            uncertain_decision_time[i] = t
            actual_decision_time[i] = time
            break

0.27165339526827836 1.0
0.011268823318335959 0.9999999999954734
0.1549639705989525 1.0
0.19865510321606714 1.0
0.3025608571245702 1.0
0.4492386035527051 1.0
0.05222422003047051 0.9999999999999999
0.10272657546505749 1.0
0.2407057079018682 1.0
0.026067230176369992 0.9999999999999719
0.11151323172867585 1.0
0.35725259001698834 1.0
0.001135920420734033 0.9999999239926323
0.4552621773651939 1.0
0.42005400048225366 1.0
0.23784586088873672 1.0
0.1075971105436833 1.0
0.3307891546270525 1.0
0.4126826510455752 1.0
0.27352113104346465 1.0
0.4211540381984219 1.0
0.31100410554555064 1.0
0.03085942518114761 0.9999999999999913
0.239925827670623 1.0
0.16906455502430737 1.0
0.4201137222399107 1.0
0.2671205427247478 1.0
0.0010935830425226054 0.9999999138591241
0.45657583190636675 1.0
0.4496715191542871 1.0
0.23193759925885726 1.0
0.13712493582068902 1.0
0.3888859458359641 1.0
0.21281882868083066 1.0
0.49763012124754014 1.0
0.41745206785060773 1.0
0.39200949724128226 1.0
0.1481539719676946 1.0
0.3401711

In [49]:
np.mean(actual_decision_time)

1065.9

### Plot expected reward over time

This plots expected reward given the agent movement time

In [ ]:
expected_reward = 